In [5]:
import streamlit as st
import tweepy as tw
import pandas as pd

In [17]:
# Get column names
api_colnames_call = api.user_timeline(screen_name = 'TSNBobMcKenzie',count=200)
json_data = [c._json for c in api_colnames_call]

# create dataframe from JSON result
api_colnames = pd.json_normalize(json_data)

# print all column names
for col in api_colnames.columns:
    print(col)

created_at
id
id_str
text
truncated
source
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
geo
coordinates
place
contributors
is_quote_status
retweet_count
favorite_count
favorited
retweeted
lang
entities.hashtags
entities.symbols
entities.user_mentions
entities.urls
user.id
user.id_str
user.name
user.screen_name
user.location
user.description
user.url
user.entities.url.urls
user.entities.description.urls
user.protected
user.followers_count
user.friends_count
user.listed_count
user.created_at
user.favourites_count
user.utc_offset
user.time_zone
user.geo_enabled
user.verified
user.statuses_count
user.lang
user.contributors_enabled
user.is_translator
user.is_translation_enabled
user.profile_background_color
user.profile_background_image_url
user.profile_background_image_url_https
user.profile_background_tile
user.profile_image_url
user.profile_image_url_https
user.profile_banner_url
user.profile_link_color
user.profile_s

In [14]:
def get_user_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    #Adapted by: https://gist.github.com/yanofsky/5436496?fbclid=IwAR12gb56FOxTNI6R3SfiwpnbPpTvKLoeGR3kP0peQ1nGilcwsF8bR0LSVqE
    
    # Set up Twitter API access
    # Define access keys and tokens
    consumer_key = st.secrets['consumer_key']
    consumer_secret = st.secrets['consumer_secret']
    access_token = st.secrets['access_token']
    access_token_secret = st.secrets['access_token_secret']

    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tw.API(auth, wait_on_rate_limit = True)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    for i in range(1):
            
            #all subsiquent requests use the max_id param to prevent duplicates
            new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
            
            #save most recent tweets
            alltweets.extend(new_tweets)
            
            #update the id of the oldest tweet less one
            oldest = alltweets[-1].id - 1
            i +=1
    
    #transform the tweepy tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text, tweet.retweet_count, tweet.favorite_count] for tweet in alltweets]

    #transform 2D array into pandas dataframe
    df_tweets = pd.DataFrame(data=outtweets, columns=['id_str', 'created_at', 'text', 'rt_count', 'fav_count'])

    return df_tweets

In [15]:
df = get_user_tweets("TSNBobMcKenzie")

In [10]:
df.head()

,id_str,created_at,text,rt_count,fav_count
0,1412003822394875907,2021-07-05 11:01:59,@barrysacks1 @provincesports @rayferrarotsn no...,0,0
1,1411704305309175815,2021-07-04 15:11:49,Congrats to @MikeMcKenzie11 on completing his ...,12,186
2,1411134094210129920,2021-07-03 01:26:00,@tsnjamesduthie First ballot Hall of Famer.,3,117
3,1411121923979984896,2021-07-03 00:37:38,@MikeMcKenzie11 Everyone is a hockey expert. 😂,0,32
4,1411084623313002500,2021-07-02 22:09:25,Substituted 8 hours of scouts interviews yeste...,25,682


In [16]:
df.shape

(400, 5)